In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('./data/train.csv')
data.head()

In [ ]:
cat_df_list = []

# Iterate over the rows of the data
for index, row in tqdm(data.iterrows()):
    cat_list = np.tile(row[:6].values, (len(row[6:]), 1)) # 첫 6개 데이터를 반복
    cat_df = pd.DataFrame(cat_list, columns=data.columns[:6]) # 반복된 데이터를 데이터프레임으로 변환
    cat_df['판매량'] = row[6:].values # 판매량 데이터 추가
    cat_df_list.append(cat_df)

cat_df = pd.concat(cat_df_list, axis=0)
cat_df.reset_index(drop=True, inplace=True)
cat_df

In [ ]:
sale_data = pd.read_csv('./data/sales.csv')
sale_data.head()

In [ ]:
cat_df_list = []

for index, row in tqdm(sale_data.iterrows()):
    cat_df_list.append(row[6:].values)

sales_amount = np.concatenate(cat_df_list).flatten()
cat_df['판매금액'] = sales_amount

cat_df.reset_index(drop=True, inplace=True)

In [ ]:
cat_df['판매량'] = cat_df['판매량'].astype(int)
cat_df['판매금액'] = cat_df['판매금액'].astype(int)

In [ ]:
cat_df['개당판매금액'] = cat_df['판매금액'] / cat_df['판매량']

In [ ]:
cat_df[cat_df['ID'] == 1]['개당판매금액'].plot()

In [ ]:
def fill_missing_values(group):
    return group.fillna(method='ffill').fillna(method='bfill')

cat_df['개당판매금액'] = cat_df.groupby('ID')['개당판매금액'].transform(fill_missing_values)

In [ ]:
cat_df[cat_df['ID'] == 1]['개당판매금액'].plot()

In [ ]:
cat_df.drop('판매금액', axis=1, inplace=True)
cat_df

In [ ]:
brand_cnt_df = pd.read_csv('./data/brand_keyword_cnt.csv')
brand_cnt_df

In [ ]:
# Using a function to apply operations on each group of unique 'ID'
tqdm.pandas()

def process_group(sub_df):
    brand_name = sub_df['브랜드'].iloc[0]
    values = brand_cnt_df.loc[brand_cnt_df['브랜드'] == brand_name].iloc[:, 1:].values.flatten()
    sub_df['언급량'] = values
    return sub_df

# Group the DataFrame by 'ID' and apply the function
cat_df = cat_df.groupby('ID').progress_apply(process_group).reset_index(drop=True)

In [ ]:
target_y = cat_df['판매량']
cat_df.drop('판매량', axis=1, inplace=True)
cat_df['판매량'] = target_y

In [ ]:
cat_df

In [ ]:
cat_df.to_csv('./data/preprocess_train_data.csv', index=False)